In [1]:
import pandas as pd
from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

/usr/local/lib/python3.5/dist-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
train = pd.read_csv('data/result_train.csv')
train =  train.drop(['Unnamed: 0', 'Unnamed: 0.1', 'isTrain'], axis=1)
train = train.dropna(subset=['lat', 'long'])

test = pd.read_csv('data/result_test.csv')
test = test.drop(['Unnamed: 0', 'Unnamed: 0.1', 'isTrain'], axis=1)

In [3]:
train = train.dropna(subset=['lat', 'long'])
y = train['target']
train = train.drop(['target'], axis=1)

In [4]:
train['isTrain'] = True
test['isTrain'] = False
X = train.append(test, sort=False)

In [5]:
from sklearn.neighbors import KNeighborsClassifier
from math import sin, cos, sqrt, atan2, radians
R = 6373.0 # радиус земли в километрах

def distance(x,y):
    """
    Параметры
    ----------
    x : tuple, широта и долгота первой геокоординаты 
    y : tuple, широта и долгота второй геокоординаты 
    
    Результат
    ----------
    result : дистанция в метрах между двумя геокоординатами
    """
    lat_a, long_a, lat_b, long_b = map(radians, [*x,*y])    
    dlon = long_b - long_a
    dlat = lat_b - lat_a
    a = sin(dlat/2)**2 + cos(lat_a) * cos(lat_b) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c * 1000

In [6]:
import numpy as np
knc = KNeighborsClassifier(metric=distance)
dots = X[['lat','long']].dropna()
X = X.drop(['lat', 'long'], axis=1)
knc.fit(X=dots , y=np.ones(dots.shape[0]))
distances, indexes = knc.kneighbors(X=dots, n_neighbors=6,)
for i in range(1,6):
    dots['distance_%s'%i] = distances[:,i]
    dots['indexes_%s'%i] = indexes[:,i]
dots['mean'] = dots.iloc[:,dots.columns.str.contains('distance')].mean(axis=1)
X = pd.concat([X, dots], axis=1)

In [7]:
import numpy as np
import reverse_geocode

def location(lat, long):
    coordinates = (lat, long), (lat, long)
    city = reverse_geocode.search(coordinates)[0]['city']
    return city
    
X['locality'] = np.vectorize(location)(X['lat'], X['long'])
one_hots = pd.get_dummies(X['locality'], dummy_na=True)
X = pd.concat([X, one_hots], axis=1)
X = X.drop(['locality'], axis=1)

In [8]:
# rare_cities = X.locality.value_counts()[(X.locality.value_counts() < 20) ==True].index
# X.locality = X.locality.apply(lambda x: 'RARE' if x in rare_cities else x)
# X.locality= X.locality.rank().fillna(-1)

In [9]:
def type_filtration(class_ru):
    class_ru_split = str(class_ru).split(' ')
    if 'Магазин' in class_ru_split or 'магазин' in class_ru_split:
        return 'магазин'
    if 'Торговый' in class_ru_split or 'Торговый' in class_ru_split:
        return 'магазин'
    return class_ru

# X['type'] = X['class_ru'].apply(type_filtration)
# X = X.drop(['class_ru'], axis=1)

one_hots = pd.get_dummies(X['class_ru'], dummy_na=True)
X = pd.concat([X, one_hots], axis=1)
X = X.drop(['class_ru'], axis=1)


In [10]:
bool_column = ['everyday', 'friday', 'monday', 'saturday', 'sunday', 'thursday', 'tuesday', 'wednesday', '24_hours']
for column in bool_column:
    X[column] = X[column].astype(int)
X = X.fillna(-1)
X = X.drop(['address', 'address_rus'], axis=1)

In [11]:
# train, valid, y_train, y_valid = train_test_split(train, y , test_size=0.5)

In [12]:
train = X[X.isTrain]
test = X[~X.isTrain]
train = train.drop(['isTrain'], axis=1)
test = test.drop(['isTrain'], axis=1)

In [13]:
import lightgbm as lgb
rfr_model = RandomForestRegressor(n_estimators=2500, n_jobs=3)
etr_model = ExtraTreesRegressor(n_estimators=2500, n_jobs=3)
abr_model = xgb.XGBRegressor(n_estimators=2500, max_depth=3, n_jobs=3)
cat_model = CatBoostRegressor(iterations=2500)
meta_model = lgb.LGBMRegressor(objective = 'regression',  
                            max_depth = 3,
                            colsample_bytre = 0.8,
                            subsample = 0.8, 
                            learning_rate = 0.1,
                            n_estimators = 2500)


In [14]:
rfr_model.fit(train, y)
etr_model.fit(train, y)
cat_model.fit(train, y)
abr_model.fit(train.values, y.values)

0:	learn: 0.0845696	total: 126ms	remaining: 5m 14s
1:	learn: 0.0830396	total: 225ms	remaining: 4m 40s
2:	learn: 0.0815766	total: 322ms	remaining: 4m 28s
3:	learn: 0.0801705	total: 434ms	remaining: 4m 30s
4:	learn: 0.0788262	total: 513ms	remaining: 4m 16s
5:	learn: 0.0775355	total: 611ms	remaining: 4m 13s
6:	learn: 0.0763036	total: 699ms	remaining: 4m 8s
7:	learn: 0.0751315	total: 804ms	remaining: 4m 10s
8:	learn: 0.0740072	total: 907ms	remaining: 4m 10s
9:	learn: 0.0729341	total: 1.01s	remaining: 4m 11s
10:	learn: 0.0719087	total: 1.09s	remaining: 4m 6s
11:	learn: 0.0709316	total: 1.18s	remaining: 4m 4s
12:	learn: 0.0699545	total: 1.26s	remaining: 4m 1s
13:	learn: 0.0690640	total: 1.36s	remaining: 4m 1s
14:	learn: 0.0681372	total: 1.45s	remaining: 4m
15:	learn: 0.0672554	total: 1.54s	remaining: 3m 58s
16:	learn: 0.0664084	total: 1.63s	remaining: 3m 57s
17:	learn: 0.0655963	total: 1.71s	remaining: 3m 55s
18:	learn: 0.0648963	total: 1.8s	remaining: 3m 55s
19:	learn: 0.0642268	total: 1.88

160:	learn: 0.0465506	total: 13.8s	remaining: 3m 21s
161:	learn: 0.0465180	total: 13.9s	remaining: 3m 21s
162:	learn: 0.0465048	total: 14s	remaining: 3m 21s
163:	learn: 0.0464866	total: 14.1s	remaining: 3m 21s
164:	learn: 0.0464778	total: 14.2s	remaining: 3m 20s
165:	learn: 0.0464704	total: 14.3s	remaining: 3m 20s
166:	learn: 0.0464543	total: 14.4s	remaining: 3m 20s
167:	learn: 0.0464321	total: 14.5s	remaining: 3m 20s
168:	learn: 0.0464089	total: 14.6s	remaining: 3m 21s
169:	learn: 0.0464037	total: 14.7s	remaining: 3m 21s
170:	learn: 0.0463981	total: 14.8s	remaining: 3m 20s
171:	learn: 0.0463917	total: 14.9s	remaining: 3m 21s
172:	learn: 0.0463597	total: 14.9s	remaining: 3m 20s
173:	learn: 0.0463452	total: 15s	remaining: 3m 20s
174:	learn: 0.0463386	total: 15.1s	remaining: 3m 21s
175:	learn: 0.0463290	total: 15.2s	remaining: 3m 20s
176:	learn: 0.0463107	total: 15.3s	remaining: 3m 21s
177:	learn: 0.0463058	total: 15.4s	remaining: 3m 20s
178:	learn: 0.0462981	total: 15.5s	remaining: 3m 2

318:	learn: 0.0446945	total: 28.5s	remaining: 3m 14s
319:	learn: 0.0446670	total: 28.6s	remaining: 3m 14s
320:	learn: 0.0446530	total: 28.6s	remaining: 3m 14s
321:	learn: 0.0446363	total: 28.7s	remaining: 3m 14s
322:	learn: 0.0446271	total: 28.9s	remaining: 3m 14s
323:	learn: 0.0446183	total: 29s	remaining: 3m 14s
324:	learn: 0.0446023	total: 29.1s	remaining: 3m 14s
325:	learn: 0.0445965	total: 29.2s	remaining: 3m 14s
326:	learn: 0.0445897	total: 29.3s	remaining: 3m 14s
327:	learn: 0.0445754	total: 29.4s	remaining: 3m 14s
328:	learn: 0.0445660	total: 29.5s	remaining: 3m 14s
329:	learn: 0.0445486	total: 29.6s	remaining: 3m 14s
330:	learn: 0.0445427	total: 29.7s	remaining: 3m 14s
331:	learn: 0.0445269	total: 29.8s	remaining: 3m 14s
332:	learn: 0.0445217	total: 29.9s	remaining: 3m 14s
333:	learn: 0.0445139	total: 30s	remaining: 3m 14s
334:	learn: 0.0444951	total: 30.1s	remaining: 3m 14s
335:	learn: 0.0444912	total: 30.2s	remaining: 3m 14s
336:	learn: 0.0444867	total: 30.3s	remaining: 3m 1

476:	learn: 0.0431393	total: 44.4s	remaining: 3m 8s
477:	learn: 0.0431290	total: 44.5s	remaining: 3m 8s
478:	learn: 0.0431232	total: 44.6s	remaining: 3m 8s
479:	learn: 0.0431198	total: 44.7s	remaining: 3m 8s
480:	learn: 0.0431171	total: 44.9s	remaining: 3m 8s
481:	learn: 0.0431051	total: 45s	remaining: 3m 8s
482:	learn: 0.0430979	total: 45.1s	remaining: 3m 8s
483:	learn: 0.0430830	total: 45.2s	remaining: 3m 8s
484:	learn: 0.0430777	total: 45.3s	remaining: 3m 8s
485:	learn: 0.0430682	total: 45.4s	remaining: 3m 8s
486:	learn: 0.0430529	total: 45.5s	remaining: 3m 8s
487:	learn: 0.0430500	total: 45.6s	remaining: 3m 8s
488:	learn: 0.0430435	total: 45.8s	remaining: 3m 8s
489:	learn: 0.0430405	total: 45.9s	remaining: 3m 8s
490:	learn: 0.0430263	total: 46s	remaining: 3m 8s
491:	learn: 0.0430135	total: 46.1s	remaining: 3m 8s
492:	learn: 0.0430107	total: 46.3s	remaining: 3m 8s
493:	learn: 0.0429992	total: 46.4s	remaining: 3m 8s
494:	learn: 0.0429966	total: 46.5s	remaining: 3m 8s
495:	learn: 0.04

636:	learn: 0.0420283	total: 1m	remaining: 2m 56s
637:	learn: 0.0420228	total: 1m	remaining: 2m 56s
638:	learn: 0.0420150	total: 1m	remaining: 2m 56s
639:	learn: 0.0420108	total: 1m	remaining: 2m 56s
640:	learn: 0.0420035	total: 1m	remaining: 2m 56s
641:	learn: 0.0419882	total: 1m	remaining: 2m 55s
642:	learn: 0.0419846	total: 1m	remaining: 2m 55s
643:	learn: 0.0419794	total: 1m	remaining: 2m 55s
644:	learn: 0.0419765	total: 1m 1s	remaining: 2m 55s
645:	learn: 0.0419730	total: 1m 1s	remaining: 2m 55s
646:	learn: 0.0419700	total: 1m 1s	remaining: 2m 55s
647:	learn: 0.0419614	total: 1m 1s	remaining: 2m 55s
648:	learn: 0.0419421	total: 1m 1s	remaining: 2m 55s
649:	learn: 0.0419309	total: 1m 1s	remaining: 2m 54s
650:	learn: 0.0419264	total: 1m 1s	remaining: 2m 54s
651:	learn: 0.0419196	total: 1m 1s	remaining: 2m 54s
652:	learn: 0.0419090	total: 1m 1s	remaining: 2m 54s
653:	learn: 0.0419066	total: 1m 1s	remaining: 2m 54s
654:	learn: 0.0419032	total: 1m 1s	remaining: 2m 54s
655:	learn: 0.041

...

2009:	learn: 0.0360546	total: 3m 12s	remaining: 46.8s
2010:	learn: 0.0360524	total: 3m 12s	remaining: 46.7s
2011:	learn: 0.0360505	total: 3m 12s	remaining: 46.6s
2012:	learn: 0.0360455	total: 3m 12s	remaining: 46.5s
2013:	learn: 0.0360449	total: 3m 12s	remaining: 46.4s
2014:	learn: 0.0360410	total: 3m 12s	remaining: 46.4s
2015:	learn: 0.0360378	total: 3m 12s	remaining: 46.3s
2016:	learn: 0.0360352	total: 3m 12s	remaining: 46.2s
2017:	learn: 0.0360321	total: 3m 12s	remaining: 46.1s
2018:	learn: 0.0360274	total: 3m 12s	remaining: 46s
2019:	learn: 0.0360269	total: 3m 13s	remaining: 45.9s
2020:	learn: 0.0360213	total: 3m 13s	remaining: 45.8s
2021:	learn: 0.0360206	total: 3m 13s	remaining: 45.7s
2022:	learn: 0.0360171	total: 3m 13s	remaining: 45.6s
2023:	learn: 0.0360121	total: 3m 13s	remaining: 45.5s
2024:	learn: 0.0360045	total: 3m 13s	remaining: 45.4s
2025:	learn: 0.0360028	total: 3m 13s	remaining: 45.3s
2026:	learn: 0.0360012	total: 3m 13s	remaining: 45.2s
2027:	learn: 0.0359997	total: 

2162:	learn: 0.0355716	total: 3m 26s	remaining: 32.1s
2163:	learn: 0.0355683	total: 3m 26s	remaining: 32s
2164:	learn: 0.0355610	total: 3m 26s	remaining: 31.9s
2165:	learn: 0.0355555	total: 3m 26s	remaining: 31.8s
2166:	learn: 0.0355532	total: 3m 26s	remaining: 31.7s
2167:	learn: 0.0355490	total: 3m 26s	remaining: 31.6s
2168:	learn: 0.0355392	total: 3m 26s	remaining: 31.6s
2169:	learn: 0.0355306	total: 3m 26s	remaining: 31.5s
2170:	learn: 0.0355293	total: 3m 26s	remaining: 31.4s
2171:	learn: 0.0355246	total: 3m 27s	remaining: 31.3s
2172:	learn: 0.0355205	total: 3m 27s	remaining: 31.2s
2173:	learn: 0.0355135	total: 3m 27s	remaining: 31.1s
2174:	learn: 0.0355130	total: 3m 27s	remaining: 31s
2175:	learn: 0.0355049	total: 3m 27s	remaining: 30.9s
2176:	learn: 0.0355027	total: 3m 27s	remaining: 30.8s
2177:	learn: 0.0354979	total: 3m 27s	remaining: 30.7s
2178:	learn: 0.0354972	total: 3m 27s	remaining: 30.6s
2179:	learn: 0.0354956	total: 3m 27s	remaining: 30.5s
2180:	learn: 0.0354930	total: 3m

2315:	learn: 0.0350828	total: 3m 40s	remaining: 17.5s
2316:	learn: 0.0350816	total: 3m 40s	remaining: 17.4s
2317:	learn: 0.0350808	total: 3m 40s	remaining: 17.3s
2318:	learn: 0.0350805	total: 3m 40s	remaining: 17.2s
2319:	learn: 0.0350800	total: 3m 40s	remaining: 17.1s
2320:	learn: 0.0350778	total: 3m 40s	remaining: 17s
2321:	learn: 0.0350737	total: 3m 40s	remaining: 16.9s
2322:	learn: 0.0350721	total: 3m 40s	remaining: 16.8s
2323:	learn: 0.0350709	total: 3m 40s	remaining: 16.7s
2324:	learn: 0.0350659	total: 3m 40s	remaining: 16.6s
2325:	learn: 0.0350614	total: 3m 41s	remaining: 16.5s
2326:	learn: 0.0350576	total: 3m 41s	remaining: 16.4s
2327:	learn: 0.0350554	total: 3m 41s	remaining: 16.3s
2328:	learn: 0.0350530	total: 3m 41s	remaining: 16.2s
2329:	learn: 0.0350504	total: 3m 41s	remaining: 16.2s
2330:	learn: 0.0350438	total: 3m 41s	remaining: 16.1s
2331:	learn: 0.0350429	total: 3m 41s	remaining: 16s
2332:	learn: 0.0350383	total: 3m 41s	remaining: 15.9s
2333:	learn: 0.0350356	total: 3m

2471:	learn: 0.0345971	total: 3m 54s	remaining: 2.65s
2472:	learn: 0.0345953	total: 3m 54s	remaining: 2.56s
2473:	learn: 0.0345929	total: 3m 54s	remaining: 2.46s
2474:	learn: 0.0345920	total: 3m 54s	remaining: 2.37s
2475:	learn: 0.0345844	total: 3m 54s	remaining: 2.27s
2476:	learn: 0.0345798	total: 3m 54s	remaining: 2.18s
2477:	learn: 0.0345752	total: 3m 54s	remaining: 2.08s
2478:	learn: 0.0345716	total: 3m 54s	remaining: 1.99s
2479:	learn: 0.0345670	total: 3m 54s	remaining: 1.89s
2480:	learn: 0.0345624	total: 3m 55s	remaining: 1.8s
2481:	learn: 0.0345615	total: 3m 55s	remaining: 1.71s
2482:	learn: 0.0345580	total: 3m 55s	remaining: 1.61s
2483:	learn: 0.0345567	total: 3m 55s	remaining: 1.51s
2484:	learn: 0.0345559	total: 3m 55s	remaining: 1.42s
2485:	learn: 0.0345520	total: 3m 55s	remaining: 1.32s
2486:	learn: 0.0345501	total: 3m 55s	remaining: 1.23s
2487:	learn: 0.0345451	total: 3m 55s	remaining: 1.14s
2488:	learn: 0.0345426	total: 3m 55s	remaining: 1.04s
2489:	learn: 0.0345400	total:

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=2500,
       n_jobs=3, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [15]:
feature_imp_rfr = pd.DataFrame([rfr_model.feature_importances_], columns=train.columns)
feature_imp_etr = pd.DataFrame([etr_model.feature_importances_], columns=train.columns)
feature_imp_abr = pd.DataFrame([abr_model.feature_importances_], columns=train.columns)
feature_imp_cat = pd.DataFrame([cat_model.feature_importances_], columns=train.columns)
feature = pd.concat([feature_imp_rfr, feature_imp_etr, feature_imp_abr, feature_imp_cat], axis=0)

In [16]:
train = train[train.columns.dropna()]
test = test[train.columns.dropna()]

In [17]:
train = train.drop('index', axis=1)
test = test.drop('index', axis=1)

In [18]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(feature)

         id  atm_group     index  everyday    friday    monday  saturday  \
0  0.058025   0.632593  0.021258  0.000523  0.000472  0.000955  0.001067   
0  0.076595   0.612646  0.016485  0.001100  0.000934  0.002211  0.002505   
0  0.173138   0.022053  0.059241  0.003719  0.000346  0.001124  0.000259   
0  5.453239  68.984535  1.186490  0.010783  0.030418  0.027430  0.054960   

     sunday  thursday   tuesday  wednesday  24_hours       lat      long  \
0  0.000490  0.000337  0.000374   0.000354  0.000568  0.028164  0.044562   
0  0.002240  0.000718  0.000840   0.000698  0.001122  0.012443  0.023616   
0  0.000259  0.000173  0.000000   0.000000  0.001643  0.039523  0.031047   
0  0.065693  0.012038  0.010069   0.012093  0.045158  2.419345  5.537922   

   distance_1  indexes_1  distance_2  indexes_2  distance_3  indexes_3  \
0    0.030880   0.016289    0.012052   0.015053    0.010621   0.016125   
0    0.012135   0.009387    0.008209   0.008894    0.007363   0.008884   
0    0.014356   

In [19]:
bad_columns = []
for column in train.columns:
    if np.sum(feature[column].values) >= 0.0001:
        bad_columns.append(column)


In [20]:
for column in bad_columns:
    train = train.drop([column], axis=1)
    test = test.drop([column], axis=1)

In [21]:
del rfr_model, etr_model, cat_model, abr_model

In [22]:
import lightgbm as lgb
rfr_model = RandomForestRegressor(n_estimators=2500, n_jobs=3)
etr_model = ExtraTreesRegressor(n_estimators=2500, n_jobs=3)
abr_model = xgb.XGBRegressor(n_estimators=2500, max_depth=3, n_jobs=3)
cat_model = CatBoostRegressor(iterations=2500)
meta_model = lgb.LGBMRegressor(objective = 'regression',  
                            max_depth = 3,
                            colsample_bytre = 0.8,
                            subsample = 0.8, 
                            learning_rate = 0.1,
                            n_estimators = 2500)

In [23]:
rfr_model.fit(train, y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=2500, n_jobs=3,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [24]:
etr_model.fit(train, y)

ExtraTreesRegressor(bootstrap=False, criterion='mse', max_depth=None,
          max_features='auto', max_leaf_nodes=None,
          min_impurity_decrease=0.0, min_impurity_split=None,
          min_samples_leaf=1, min_samples_split=2,
          min_weight_fraction_leaf=0.0, n_estimators=2500, n_jobs=3,
          oob_score=False, random_state=None, verbose=0, warm_start=False)

In [25]:
cat_model.fit(train, y)

0:	learn: 0.0862149	total: 43.9ms	remaining: 1m 49s
1:	learn: 0.0862119	total: 87.6ms	remaining: 1m 49s
2:	learn: 0.0862106	total: 131ms	remaining: 1m 48s
3:	learn: 0.0862096	total: 174ms	remaining: 1m 48s
4:	learn: 0.0862037	total: 209ms	remaining: 1m 44s
5:	learn: 0.0862011	total: 247ms	remaining: 1m 42s
6:	learn: 0.0861991	total: 283ms	remaining: 1m 40s
7:	learn: 0.0861926	total: 326ms	remaining: 1m 41s
8:	learn: 0.0861896	total: 372ms	remaining: 1m 42s
9:	learn: 0.0861871	total: 423ms	remaining: 1m 45s
10:	learn: 0.0861864	total: 468ms	remaining: 1m 45s
11:	learn: 0.0861857	total: 511ms	remaining: 1m 45s
12:	learn: 0.0861835	total: 547ms	remaining: 1m 44s
13:	learn: 0.0861803	total: 591ms	remaining: 1m 44s
14:	learn: 0.0861780	total: 627ms	remaining: 1m 43s
15:	learn: 0.0861763	total: 671ms	remaining: 1m 44s
16:	learn: 0.0861736	total: 714ms	remaining: 1m 44s
17:	learn: 0.0861703	total: 758ms	remaining: 1m 44s
18:	learn: 0.0861666	total: 805ms	remaining: 1m 45s
19:	learn: 0.0861632

157:	learn: 0.0858188	total: 6.54s	remaining: 1m 37s
158:	learn: 0.0858167	total: 6.59s	remaining: 1m 37s
159:	learn: 0.0858147	total: 6.63s	remaining: 1m 36s
160:	learn: 0.0858125	total: 6.68s	remaining: 1m 36s
161:	learn: 0.0858098	total: 6.72s	remaining: 1m 36s
162:	learn: 0.0858088	total: 6.75s	remaining: 1m 36s
163:	learn: 0.0858077	total: 6.79s	remaining: 1m 36s
164:	learn: 0.0858046	total: 6.84s	remaining: 1m 36s
165:	learn: 0.0858035	total: 6.88s	remaining: 1m 36s
166:	learn: 0.0858022	total: 6.92s	remaining: 1m 36s
167:	learn: 0.0857999	total: 6.96s	remaining: 1m 36s
168:	learn: 0.0857982	total: 7s	remaining: 1m 36s
169:	learn: 0.0857967	total: 7.05s	remaining: 1m 36s
170:	learn: 0.0857950	total: 7.09s	remaining: 1m 36s
171:	learn: 0.0857941	total: 7.13s	remaining: 1m 36s
172:	learn: 0.0857928	total: 7.17s	remaining: 1m 36s
173:	learn: 0.0857918	total: 7.21s	remaining: 1m 36s
174:	learn: 0.0857899	total: 7.25s	remaining: 1m 36s
175:	learn: 0.0857876	total: 7.28s	remaining: 1m 

314:	learn: 0.0855781	total: 13s	remaining: 1m 29s
315:	learn: 0.0855759	total: 13s	remaining: 1m 29s
316:	learn: 0.0855737	total: 13s	remaining: 1m 29s
317:	learn: 0.0855725	total: 13.1s	remaining: 1m 29s
318:	learn: 0.0855709	total: 13.1s	remaining: 1m 29s
319:	learn: 0.0855693	total: 13.2s	remaining: 1m 29s
320:	learn: 0.0855672	total: 13.2s	remaining: 1m 29s
321:	learn: 0.0855652	total: 13.3s	remaining: 1m 29s
322:	learn: 0.0855636	total: 13.3s	remaining: 1m 29s
323:	learn: 0.0855611	total: 13.4s	remaining: 1m 29s
324:	learn: 0.0855596	total: 13.4s	remaining: 1m 29s
325:	learn: 0.0855569	total: 13.4s	remaining: 1m 29s
326:	learn: 0.0855550	total: 13.5s	remaining: 1m 29s
327:	learn: 0.0855520	total: 13.5s	remaining: 1m 29s
328:	learn: 0.0855512	total: 13.5s	remaining: 1m 29s
329:	learn: 0.0855493	total: 13.6s	remaining: 1m 29s
330:	learn: 0.0855477	total: 13.6s	remaining: 1m 29s
331:	learn: 0.0855466	total: 13.7s	remaining: 1m 29s
332:	learn: 0.0855447	total: 13.7s	remaining: 1m 29s

472:	learn: 0.0853443	total: 19.5s	remaining: 1m 23s
473:	learn: 0.0853435	total: 19.6s	remaining: 1m 23s
474:	learn: 0.0853419	total: 19.6s	remaining: 1m 23s
475:	learn: 0.0853409	total: 19.6s	remaining: 1m 23s
476:	learn: 0.0853397	total: 19.7s	remaining: 1m 23s
477:	learn: 0.0853383	total: 19.7s	remaining: 1m 23s
478:	learn: 0.0853373	total: 19.8s	remaining: 1m 23s
479:	learn: 0.0853361	total: 19.8s	remaining: 1m 23s
480:	learn: 0.0853343	total: 19.8s	remaining: 1m 23s
481:	learn: 0.0853327	total: 19.9s	remaining: 1m 23s
482:	learn: 0.0853311	total: 19.9s	remaining: 1m 23s
483:	learn: 0.0853302	total: 20s	remaining: 1m 23s
484:	learn: 0.0853293	total: 20s	remaining: 1m 23s
485:	learn: 0.0853281	total: 20.1s	remaining: 1m 23s
486:	learn: 0.0853267	total: 20.1s	remaining: 1m 23s
487:	learn: 0.0853250	total: 20.1s	remaining: 1m 23s
488:	learn: 0.0853243	total: 20.2s	remaining: 1m 23s
489:	learn: 0.0853234	total: 20.2s	remaining: 1m 22s
490:	learn: 0.0853226	total: 20.3s	remaining: 1m 2

631:	learn: 0.0851935	total: 26.1s	remaining: 1m 17s
632:	learn: 0.0851921	total: 26.2s	remaining: 1m 17s
633:	learn: 0.0851912	total: 26.2s	remaining: 1m 17s
634:	learn: 0.0851898	total: 26.3s	remaining: 1m 17s
635:	learn: 0.0851891	total: 26.3s	remaining: 1m 17s
636:	learn: 0.0851886	total: 26.3s	remaining: 1m 17s
637:	learn: 0.0851875	total: 26.4s	remaining: 1m 16s
638:	learn: 0.0851867	total: 26.4s	remaining: 1m 16s
639:	learn: 0.0851860	total: 26.5s	remaining: 1m 16s
640:	learn: 0.0851852	total: 26.5s	remaining: 1m 16s
641:	learn: 0.0851846	total: 26.5s	remaining: 1m 16s
642:	learn: 0.0851842	total: 26.6s	remaining: 1m 16s
643:	learn: 0.0851832	total: 26.6s	remaining: 1m 16s
644:	learn: 0.0851826	total: 26.7s	remaining: 1m 16s
645:	learn: 0.0851819	total: 26.7s	remaining: 1m 16s
646:	learn: 0.0851813	total: 26.8s	remaining: 1m 16s
647:	learn: 0.0851802	total: 26.8s	remaining: 1m 16s
648:	learn: 0.0851796	total: 26.8s	remaining: 1m 16s
649:	learn: 0.0851788	total: 26.9s	remaining: 

...

1742:	learn: 0.0846623	total: 1m 12s	remaining: 31.3s
1743:	learn: 0.0846622	total: 1m 12s	remaining: 31.3s
1744:	learn: 0.0846620	total: 1m 12s	remaining: 31.3s
1745:	learn: 0.0846615	total: 1m 12s	remaining: 31.2s
1746:	learn: 0.0846607	total: 1m 12s	remaining: 31.2s
1747:	learn: 0.0846604	total: 1m 12s	remaining: 31.1s
1748:	learn: 0.0846600	total: 1m 12s	remaining: 31.1s
1749:	learn: 0.0846595	total: 1m 12s	remaining: 31.1s
1750:	learn: 0.0846591	total: 1m 12s	remaining: 31s
1751:	learn: 0.0846586	total: 1m 12s	remaining: 31s
1752:	learn: 0.0846585	total: 1m 12s	remaining: 30.9s
1753:	learn: 0.0846579	total: 1m 12s	remaining: 30.9s
1754:	learn: 0.0846575	total: 1m 12s	remaining: 30.8s
1755:	learn: 0.0846572	total: 1m 12s	remaining: 30.8s
1756:	learn: 0.0846567	total: 1m 12s	remaining: 30.8s
1757:	learn: 0.0846567	total: 1m 12s	remaining: 30.7s
1758:	learn: 0.0846558	total: 1m 12s	remaining: 30.7s
1759:	learn: 0.0846558	total: 1m 12s	remaining: 30.6s
1760:	learn: 0.0846551	total: 1m

1897:	learn: 0.0846144	total: 1m 18s	remaining: 24.9s
1898:	learn: 0.0846143	total: 1m 18s	remaining: 24.9s
1899:	learn: 0.0846139	total: 1m 18s	remaining: 24.8s
1900:	learn: 0.0846135	total: 1m 18s	remaining: 24.8s
1901:	learn: 0.0846134	total: 1m 18s	remaining: 24.8s
1902:	learn: 0.0846129	total: 1m 18s	remaining: 24.7s
1903:	learn: 0.0846125	total: 1m 18s	remaining: 24.7s
1904:	learn: 0.0846124	total: 1m 18s	remaining: 24.6s
1905:	learn: 0.0846119	total: 1m 18s	remaining: 24.6s
1906:	learn: 0.0846118	total: 1m 18s	remaining: 24.6s
1907:	learn: 0.0846116	total: 1m 19s	remaining: 24.5s
1908:	learn: 0.0846115	total: 1m 19s	remaining: 24.5s
1909:	learn: 0.0846111	total: 1m 19s	remaining: 24.4s
1910:	learn: 0.0846110	total: 1m 19s	remaining: 24.4s
1911:	learn: 0.0846106	total: 1m 19s	remaining: 24.4s
1912:	learn: 0.0846105	total: 1m 19s	remaining: 24.3s
1913:	learn: 0.0846100	total: 1m 19s	remaining: 24.3s
1914:	learn: 0.0846095	total: 1m 19s	remaining: 24.2s
1915:	learn: 0.0846093	total

2054:	learn: 0.0845680	total: 1m 25s	remaining: 18.4s
2055:	learn: 0.0845678	total: 1m 25s	remaining: 18.4s
2056:	learn: 0.0845676	total: 1m 25s	remaining: 18.3s
2057:	learn: 0.0845676	total: 1m 25s	remaining: 18.3s
2058:	learn: 0.0845672	total: 1m 25s	remaining: 18.2s
2059:	learn: 0.0845671	total: 1m 25s	remaining: 18.2s
2060:	learn: 0.0845669	total: 1m 25s	remaining: 18.2s
2061:	learn: 0.0845667	total: 1m 25s	remaining: 18.1s
2062:	learn: 0.0845667	total: 1m 25s	remaining: 18.1s
2063:	learn: 0.0845666	total: 1m 25s	remaining: 18s
2064:	learn: 0.0845665	total: 1m 25s	remaining: 18s
2065:	learn: 0.0845664	total: 1m 25s	remaining: 18s
2066:	learn: 0.0845661	total: 1m 25s	remaining: 17.9s
2067:	learn: 0.0845660	total: 1m 25s	remaining: 17.9s
2068:	learn: 0.0845656	total: 1m 25s	remaining: 17.8s
2069:	learn: 0.0845654	total: 1m 25s	remaining: 17.8s
2070:	learn: 0.0845650	total: 1m 25s	remaining: 17.7s
2071:	learn: 0.0845649	total: 1m 25s	remaining: 17.7s
2072:	learn: 0.0845645	total: 1m 2

2211:	learn: 0.0845225	total: 1m 31s	remaining: 11.9s
2212:	learn: 0.0845222	total: 1m 31s	remaining: 11.9s
2213:	learn: 0.0845220	total: 1m 31s	remaining: 11.8s
2214:	learn: 0.0845218	total: 1m 31s	remaining: 11.8s
2215:	learn: 0.0845215	total: 1m 31s	remaining: 11.8s
2216:	learn: 0.0845212	total: 1m 31s	remaining: 11.7s
2217:	learn: 0.0845210	total: 1m 31s	remaining: 11.7s
2218:	learn: 0.0845208	total: 1m 31s	remaining: 11.6s
2219:	learn: 0.0845204	total: 1m 31s	remaining: 11.6s
2220:	learn: 0.0845204	total: 1m 31s	remaining: 11.5s
2221:	learn: 0.0845203	total: 1m 31s	remaining: 11.5s
2222:	learn: 0.0845197	total: 1m 32s	remaining: 11.5s
2223:	learn: 0.0845194	total: 1m 32s	remaining: 11.4s
2224:	learn: 0.0845193	total: 1m 32s	remaining: 11.4s
2225:	learn: 0.0845191	total: 1m 32s	remaining: 11.3s
2226:	learn: 0.0845190	total: 1m 32s	remaining: 11.3s
2227:	learn: 0.0845188	total: 1m 32s	remaining: 11.3s
2228:	learn: 0.0845183	total: 1m 32s	remaining: 11.2s
2229:	learn: 0.0845178	total

2364:	learn: 0.0844812	total: 1m 37s	remaining: 5.59s
2365:	learn: 0.0844811	total: 1m 37s	remaining: 5.54s
2366:	learn: 0.0844807	total: 1m 37s	remaining: 5.5s
2367:	learn: 0.0844803	total: 1m 38s	remaining: 5.46s
2368:	learn: 0.0844801	total: 1m 38s	remaining: 5.42s
2369:	learn: 0.0844797	total: 1m 38s	remaining: 5.38s
2370:	learn: 0.0844795	total: 1m 38s	remaining: 5.34s
2371:	learn: 0.0844794	total: 1m 38s	remaining: 5.3s
2372:	learn: 0.0844793	total: 1m 38s	remaining: 5.25s
2373:	learn: 0.0844789	total: 1m 38s	remaining: 5.21s
2374:	learn: 0.0844787	total: 1m 38s	remaining: 5.17s
2375:	learn: 0.0844786	total: 1m 38s	remaining: 5.13s
2376:	learn: 0.0844785	total: 1m 38s	remaining: 5.09s
2377:	learn: 0.0844781	total: 1m 38s	remaining: 5.05s
2378:	learn: 0.0844776	total: 1m 38s	remaining: 5.01s
2379:	learn: 0.0844775	total: 1m 38s	remaining: 4.96s
2380:	learn: 0.0844773	total: 1m 38s	remaining: 4.92s
2381:	learn: 0.0844769	total: 1m 38s	remaining: 4.88s
2382:	learn: 0.0844768	total: 

In [26]:
abr_model.fit(train.values, y.values)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=2500,
       n_jobs=3, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [27]:
rfr_predict = rfr_model.predict(train)
etr_predict = etr_model.predict(train)
cat_predict = cat_model.predict(train)
abr_predict = abr_model.predict(train.values)

In [28]:
test_rfr_predict = rfr_model.predict(test)
test_etr_predict = etr_model.predict(test)
test_cat_predict = cat_model.predict(test)
test_abr_predict = abr_model.predict(test.values)

In [29]:
import numpy as np

In [30]:
stacked_preds = np.column_stack((rfr_predict, etr_predict, abr_predict, cat_predict))

In [31]:
stacked_test_preds = np.column_stack((test_rfr_predict, test_etr_predict, test_abr_predict, test_cat_predict))

In [32]:
meta_model.fit(stacked_preds, y)

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytre=0.8,
       colsample_bytree=1.0, importance_type='split', learning_rate=0.1,
       max_depth=3, min_child_samples=20, min_child_weight=0.001,
       min_split_gain=0.0, n_estimators=2500, n_jobs=-1, num_leaves=31,
       objective='regression', random_state=None, reg_alpha=0.0,
       reg_lambda=0.0, silent=True, subsample=0.8,
       subsample_for_bin=200000, subsample_freq=0)

In [33]:
final_predict = meta_model.predict(stacked_test_preds)

In [34]:
submit = pd.DataFrame(final_predict, index=test.index,columns=['target'])

In [35]:
submit.to_csv('data/submit.csv')

In [36]:
meta_model.feature_importances_

array([3227, 2209, 1627, 2715])